### Empirical Example for RobustiPY (Type 5)!

This example is designed to show how RobustiPy can be estimated with multiple dependent variables. In particular, it replicates the wonderful paper by [Amy Orben & Andrew K. Przybylski in Nature Human Behaviour, 2019](https://www.nature.com/articles/s41562-018-0506-1). Data comes from the UK Data Service [here](https://beta.ukdataservice.ac.uk/datacatalogue/series/series?id=2000031). Code comes from Amy's GitHub page, available [here](https://github.com/OrbenAmy/NHB_2019/tree/master).


In [1]:
import pandas as pd
from robustipy.models import OLSRobust
import warnings
from robustipy.prototypes import MissingValueWarning
from robustipy.utils import concat_results
from copy import deepcopy


# Ignore all MissingValueWarning warnings from robustipy
warnings.filterwarnings(
    action="ignore",
    category=MissingValueWarning,
    module=r"robustipy\.prototypes"
)


n_draws = 500
y_sample= 250

In [2]:
## lists of predictors
predictors = [
    ["fctvho00r"],
    ["fccomh00r"],
    ["fccmex00r"],
    ["fcinth00r"],
    ["fcsome00r"],
    ["tech"]
]
controls = [
    "edumot",
    "fd06e00",
    "clpar",
    "fcpaab00",
    "fpwrdscm",
    "fdacaq00",
    "fd05s00",
    "fpclsi00",
    "fpchti00",
    "fdkessl",
    "fdtots00",
    "foede000"
]

In [3]:
data = pd.read_csv('../data/nhb_2019/1_3_prep_mcs_data.csv', low_memory=False)

In [4]:
def patch_for_merge(results):
    n_specs = results.summary_df.shape[0]
    patched = deepcopy(results)
    x_label  = ", ".join(x) if isinstance(x, list) else str(x)
    y_labels = [f"composite_y_{i+1}" for i in range(n_specs)]
    if len(patched.all_b) == 1:
        patched.all_b = patched.all_b * n_specs
    if len(patched.all_p) == 1:
        patched.all_p = patched.all_p * n_specs
    patched.__dict__.pop("_y_name", None)
    patched.__dict__.pop("_x_name", None)
    patched.y_name = y_labels
    patched.x_name = [x_label] * n_specs
    patched.all_predictors = [x_label] * n_specs
    patched.specs_names = pd.Series(list(range(n_specs)))
    return patched

## No-Controls, participant

In [ ]:
y = [
    "fcmdsa00r", "fcmdsb00r", "fcmdsc00r", "fcmdsd00r", "fcmdse00r", "fcmdsf00r",
    "fcmdsg00r", "fcmdsh00r", "fcmdsi00r", "fcmdsj00r", "fcmdsk00r", "fcmdsl00r",
    "fcmdsm00r", "fcsati00r", "fcgdql00r", "fcdowl00r", "fcvalu00r", "fcgdsf00r",
    "fcscwk00r", "fcwylk00r", "fcfmly00r", "fcfrns00r", "fcschl00r", "fclife00r"
]
merged_results = None
for x in predictors:
    model = OLSRobust(y=y, x=x, data=data)
    model.fit(
        controls=[],
        draws=n_draws,
        composite_sample=y_sample,
        rescale_x=True,
        rescale_z=True,
        kfold=10,
        oos_metric="rmse",
        n_cpu=24,
    )
    patched = patch_for_merge(model.get_results())
    if merged_results is None:
        merged_results = concat_results([patched], de_dupe=False)
    else:
        merged_results = concat_results([merged_results, patched], de_dupe=False)

Calculating Composite Ys


Output()

OLSRobust is running with n_cpu=24, draws=500, folds=10, seed=None.
We're evaluating our out-of-sample predictions with the rmse metric.
The estimand of interest is fctvho00r. Let's begin the calculations...


Calculating Composite Ys


Output()

OLSRobust is running with n_cpu=24, draws=500, folds=10, seed=None.
We're evaluating our out-of-sample predictions with the rmse metric.
The estimand of interest is fccomh00r. Let's begin the calculations...


## Control, Participant

In [ ]:
for x in predictors:
    model = OLSRobust(
        y=y,
        x=x + controls,
        data=data
    )
    model.fit(
        controls=[],
        draws=n_draws,
        composite_sample=y_sample,
        rescale_x=True,
        rescale_z=True,
        kfold=10,
        oos_metric="rmse",
        n_cpu=24,
    )
    patched = patch_for_merge(model.get_results())
    merged_results = concat_results([merged_results, patched], de_dupe=False)

## No Controls, Primary carers

In [ ]:
y = [
    "fpsdpf00", "fpsdro00", "fpsdhs00", "fpsdsr00", "fpsdtt00", "fpsdsp00",
    "fpsdor00", "fpsdmw00", "fpsdhu00", "fpsdfs00", "fpsdgf00", "fpsdfb00",
    "fpsdud00", "fpsdlc00", "fpsddc00", "fpsdnc00", "fpsdky00", "fpsdoa00",
    "fpsdpb00", "fpsdvh00", "fpsdst00", "fpsdcs00", "fpsdgb00", "fpsdfe00",
    "fpsdte00",
]

for x in predictors:
    model = OLSRobust(
        y=y,
        x=x,
        data=data
    )
    model.fit(
        controls=[],
        draws=n_draws,
        composite_sample=y_sample,
        rescale_x=True,
        rescale_z=True,
        kfold=10,
        oos_metric="rmse",
        n_cpu=24,
    )
    patched = patch_for_merge(model.get_results())
    merged_results = concat_results([merged_results, patched], de_dupe=False)

## Controls, Primary Carers

In [ ]:
for x in predictors:
    model = OLSRobust(
        y=y,
        x=x+controls,
        data=data
    )
    model.fit(
        controls=[],
        draws=n_draws,
        composite_sample=y_sample,
        rescale_x=True,
        rescale_z=True,
        kfold=10,
        oos_metric="rmse",
        n_cpu=24,
    )
    patched = patch_for_merge(model.get_results())
    merged_results = concat_results([merged_results, patched], de_dupe=False)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from robustipy.figures import plot_curve, plot_hexbin_r2, plot_bdist

fig = plt.figure(figsize=(12, 6))
gs = GridSpec(6, 24, wspace=-.25, hspace=5)
ax1 = fig.add_subplot(gs[0:6, 0:14])
ax2 = fig.add_subplot(gs[0:3, 15:24])
ax3 = fig.add_subplot(gs[3:6, 15:24])
plot_curve(results_object=merged_results, loess=True,
           ci=1, ax=ax1,
           title='a.', highlights=False, inset=False)
plot_hexbin_r2(merged_results, ax2, fig,
               colormap='Spectral_r', title='b.', side='right',
              oddsratio=False)
plot_bdist(results_object=merged_results,
           ax=ax3, oddsratio=False,
           title='c.', despine_left=True, highlights=False,
           legend_bool=False)


info_text = (
    f'Original Median: –0.005\n'
    f'RobustiPy Median: {merged_results.estimates.stack().median():.3f}\n'
    f'RobustiPy Min: {merged_results.estimates.stack().min():.3f}\n'
    f'RobustiPy Max: {merged_results.estimates.stack().max():.3f}'
)
ax1.text(0.05, 0.95, info_text, transform=ax1.transAxes, va='top', ha='left',
         fontsize=10, color='black', bbox=dict(facecolor='white',
                                           edgecolor='black', boxstyle='round,pad=1'));
plt.savefig("../figures/nhb_2019/nhb_2019.pdf", bbox_inches='tight')
plt.savefig("../figures/nhb_2019/nhb_2019.png", bbox_inches='tight', dpi=400)
plt.savefig("../figures/nhb_2019/nhb_2019.svg", bbox_inches='tight')

In [ ]:
merged_results.estimates.stack().median()